In [7]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
import pickle
from sklearn.manifold import TSNE, LocallyLinearEmbedding
from scipy.spatial.distance import pdist, squareform
from scipy import signal
# import skvideo.io
from openTSNE import TSNE as oTSNE
from openTSNE import  affinity
import math
import cv2
import os
from tqdm.notebook import trange, tqdm
# plt.switch_backend('Agg')


In [8]:
def calc_dist(data):
    dist = 1 - np.abs(np.corrcoef(data, rowvar=True))
    # eliminating pixels with nan distance. These are pixels who have not changed across all images. That is std = 0
    eleminated_pixel_idx = np.all(np.isnan(dist), axis=0)
    dist = dist[:, ~np.all(np.isnan(dist), axis=0)]
    dist = dist[~np.all(np.isnan(dist), axis=1), :]
    return dist, eleminated_pixel_idx

def make_framer(frames, count= -1):
    count = [-1]
    def make_frame(t):
        count[0] += 1
        return frames[count[0]]
    return make_frame  

In [11]:
from moviepy.editor import VideoFileClip
from moviepy.editor import VideoClip
# wiggle_clip = VideoFileClip("wiggling_1.3gp")
# wiggle_clip = VideoFileClip("2023_A_scroll_scrolling_h2d4_optional_video_3.mp4")
# wiggle_clip = VideoFileClip("pexels_videos_4573 (1080p).mp4")
# wiggle_clip = VideoFileClip("video (2160p).mp4")
# wiggle_clip = VideoFileClip("fire_video_(1080p).mp4")
wiggle_clip = VideoFileClip("fire2.mp4")

# wiggle_animation = VideoClip(wiggle_clip.get_frame, duration=7)
# wiggle_animation.ipython_display(fps=15, loop=True, autoplay=True)

In [12]:
wiggle_clip_resized = wiggle_clip.resize((144, 108))
wiggle_animation = VideoClip(wiggle_clip_resized.get_frame, duration=15)
wiggle_animation.ipython_display(fps=15, loop=True, autoplay=True)

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


In [13]:
x1 = 90
y1 = 30
cropped_clip = wiggle_clip_resized.crop(x1, y1, x1+32, y1+32)
cropped_clip.size

(32, 32)

In [14]:
wiggle_frames = np.stack([f for f in cropped_clip.iter_frames()], axis=0)
make_frame = make_framer(wiggle_frames)
animation = VideoClip(make_frame, duration=10)
animation.ipython_display(fps=20, loop=True, autoplay=True)
# print('Num frames: ', len(wiggle_frames))

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


In [15]:
print('Num frames: ', len(wiggle_frames))

Num frames:  275


In [16]:
wiggle_frames_grey = np.mean(wiggle_frames, axis=-1)
wiggle_frames_grey = wiggle_frames_grey.reshape(wiggle_frames_grey.shape[0], -1)
print(wiggle_frames_grey.shape)
wiggle_dist_mat, bad_pixel_idx = calc_dist(wiggle_frames_grey[0:].T)
print('dist mat shape: ', wiggle_dist_mat.shape)

(275, 1024)
dist mat shape:  (1024, 1024)


In [17]:
steps = 200
# dist_mat_copy = dist_mat.copy()
dist_mat_copy = wiggle_dist_mat.copy()
frames = []
positions_embd = oTSNE(n_components=2, verbose=False, metric="precomputed", perplexity=4.0, initialization='random', n_iter=1).prepare_initial(dist_mat_copy)
for i in trange(steps):
    fig, ax = plt.subplots(1, 1, sharey=False, figsize=(1.5, 1.5), dpi=200)
    # ax.scatter(positions_embd[:, 0], positions_embd[:, 1], s=4, c=test_image[~bad_pixel_idx])
    frame_colors = wiggle_frames[i+0].reshape(-1, 3) / 255 
    ax.scatter(positions_embd[~bad_pixel_idx, 0], positions_embd[~bad_pixel_idx, 1], s=4, c=frame_colors[~bad_pixel_idx], alpha=0.5)
    ax.set_xlim(-3, 3)
    ax.set_ylim(-4, 4)
    ax.set_facecolor("black")
    fig.canvas.draw()
    plot_data = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    frames.append(plot_data.reshape(fig.canvas.get_width_height()[::-1] + (3,)))
    plt.close()
    
    wiggle_dist_mat_new, bad_pixel_idx = calc_dist(wiggle_frames_grey[0+2*i:0+2*i+200].T)
    positions_embd = oTSNE(n_components=2, verbose=False, metric="precomputed", perplexity=16.0, initialization=positions_embd).prepare_initial(wiggle_dist_mat_new)    
    positions_embd = positions_embd.optimize(n_iter=5, inplace=False, exaggeration=6)
    # dist_mat_copy = dist_evolution(dist_mat_copy, i) if i < 250 else dist_mat.copy()

  0%|          | 0/200 [00:00<?, ?it/s]

Standard deviation of embedding is greater than 0.0001. Initial embeddings with high variance may have display poor convergence.
Standard deviation of embedding is greater than 0.0001. Initial embeddings with high variance may have display poor convergence.
Standard deviation of embedding is greater than 0.0001. Initial embeddings with high variance may have display poor convergence.
Standard deviation of embedding is greater than 0.0001. Initial embeddings with high variance may have display poor convergence.
Standard deviation of embedding is greater than 0.0001. Initial embeddings with high variance may have display poor convergence.
Standard deviation of embedding is greater than 0.0001. Initial embeddings with high variance may have display poor convergence.
Standard deviation of embedding is greater than 0.0001. Initial embeddings with high variance may have display poor convergence.
Standard deviation of embedding is greater than 0.0001. Initial embeddings with high variance may

ValueError: The provided initialization contains a different number of points (1024) than the data provided (1013).

In [19]:
len(frames)

137

In [22]:
wiggle_frames_grey.T.shape

(1024, 275)

In [21]:
i

136

In [123]:
fps = 15
make_frame = make_framer(frames)
animation = VideoClip(make_frame, duration=(len(frames) - 1)//fps)
animation.ipython_display(fps=fps, loop=True, autoplay=True)

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


In [3]:
import cv2
from IPython.display import display, Image
import ipywidgets as widgets
import threading
# Stop button
# ================
stopButton = widgets.ToggleButton(
    value=False,
    description='Stop',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='square' # (FontAwesome names without the `fa-` prefix)
)


# Display function
# ================
def view(button):
    cap = cv2.VideoCapture(0)
    display_handle=display(None, display_id=True)
    i = 0
    while True:
        _, frame = cap.read()
        frame = cv2.flip(frame, 1) # if your camera reverses your image
        _, frame = cv2.imencode('.jpeg', frame)
        display_handle.update(Image(data=frame.tobytes()))
        if stopButton.value==True:
            cap.release()
            display_handle.update(None)

            
# Run
# ================
display(stopButton)
thread = threading.Thread(target=view, args=(stopButton,))
thread.start()

ToggleButton(value=False, button_style='danger', description='Stop', icon='square', tooltip='Description')

[ WARN:1@26.364] global /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


None